# <font color=black> Time-series features </font>
<hr style="border:1px solid black">

## <font color=#B14263> Imports

In [ ]:
import sys,json, os, glob
import pandas as pd
import numpy as np
import nibabel as nib
from scipy.stats import spearmanr

main_dir='/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project/'
sys.path.append(main_dir+'/code/')

from connectivity.ts_features import FeatureSets
from connectivity.seed_to_target import Seed2target

#statistics
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from brsc_statistics import Statistics 
from sim_matrix import Matrix
from scipy.stats import shapiro

#plotting
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from plotting import Plotting
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

config_file=main_dir + '/config/analyses/brsc_functional.json'


%matplotlib inline
%load_ext autoreload
%autoreload 2

### load config file
with open(config_file) as config_f:
    config = json.load(config_f) # load config info
stat_func=Statistics(config=config,ana_dir="",analysis="")
output_dir=config["project_dir"] +config['ts_features']["analysis_dir"]["spinalcord"]
plot=Plotting(config_file,output_dir)

<hr style="border:1px solid black">

## <font color=#B14263> A. Distribution of the dynamic time-series profiles
#### <font color=#B14263> A.1 Initialize the analysis and load the timeseries

In [ ]:
# Load config file ------------------------------------------------------------
with open(config_file) as config_f: # the notebook should be in 'xx/notebook/' folder #config_proprio
    config = json.load(config_f)


IDs=config["participants_IDs_ALL"]
structure="spinalcord"
ts_features=FeatureSets(config,IDs=IDs,structure=structure)
seed2seed=Seed2target(config_file,IDs=IDs,ana="ts_features",seed_kind="atlas",structure=[structure],verbose=0) # initialize the function


#### <font color=#B14263> A.2 Load results
Load individual csv files

In [ ]:
main_ana_dir=config["project_dir"] + config['ts_features']['analysis_dir']['spinalcord'] 
no_z_data=[];z_data=[];
for ID_nb, ID in enumerate(IDs):
    no_z_data.append(pd.read_csv(glob.glob(f'{main_ana_dir}/1_first_level/catch24/sub_{ID}*catch24.csv')[0]))
    
    
no_z_data_all = pd.concat([df for df in no_z_data], ignore_index=True)


#### <font color=#B14263> A.3 Plot distribution

In [ ]:
output_dir=config["project_dir"] +config['ts_features']["analysis_dir"]["spinalcord"]

# Feature selection
selected_cols = np.r_[8:33]
col_selected = no_z_data_all.columns[selected_cols]

# Plot layout
n_features = len(col_selected)
plots_per_row = 5
n_rows = int(np.ceil(n_features / plots_per_row))


for df in [no_z_data_all]:
    highlight_features = []
    fig, axes = plt.subplots(n_rows, plots_per_row, figsize=(4 * plots_per_row, 4 * n_rows), sharey=False)
    axes = axes.flatten()

    #Check if there is a dominant value and plot feature distribution
    for i, col in enumerate(col_selected):
        data = df[col].dropna()
        value_counts = data.value_counts()
        most_freq_count = value_counts.iloc[0] if len(value_counts) > 0 else 0 # Count the most frequent value and its count
        threshold = len(no_z_data_all) / 4  # (threshold i.e. 25%)
        f = most_freq_count / len(no_z_data_all) * 100
        
        # Plot distribution with KDE
        if (most_freq_count > threshold):
            highlight_features.append(col)
        color = "#FFAC1C" if col in highlight_features else "#8C0041" # Determine color based on highlight
        sns.histplot(data=data, kde=True, ax=axes[i],
                     bins=20, color=color, edgecolor="white")
        axes[i].set_title(col, fontsize=10)
        axes[i].set_xlabel("Feature value")
        axes[i].set_ylabel("Count")

        axes[i].text(0.95, 0.95, f"f_dom = {f:.3f} %", fontsize=8,
                     verticalalignment='top', horizontalalignment='right',
                     transform=axes[i].transAxes,
                     bbox=dict(boxstyle="round,pad=0.3", facecolor="white", edgecolor="gray", alpha=0.5))


    # Turn off unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()
    #plt.savefig(output_dir  + '/figures/catch22_distrib.png', format='png')

    print("")  # print blank line to separate outputs if needed
